# get numeration word2num response from gemini

Install necessary packages

In [1]:
# !pip install -q -U google-generativeai
# !pip install python-dotenv

In [2]:
import os
from dotenv import load_dotenv
import random
import time
import json
import google.generativeai as genai

### load model

In [3]:
load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [4]:
gemini = genai.GenerativeModel('gemini-pro')

### load data and create initial prompt

In [5]:
with open('../../data/numeration_word2num.json','r') as file:
    data = json.load(file)

# create batches, batch sizes are heuristics
# batch sizes, although input token limit is 30,720 tokens, output token limit is 2048
# which restricts the batch size
    
size_dict = {'d1':10,
             'd2':10, # big sizes are not working, model gives blank outputs
             'd3':20,
             'd4':20,
             'd5':20,
             'd6':10,
             'd7':10,}

batch_prompts = []

for datatype in data:
    # dataype is positive int, negative int, postive dec, negative dec
    for num_digit in data[datatype]:
        # num_digit is d1, d2, d3, d4, d5, d6, d7
        list_data = data[datatype][num_digit]
        # get batch size
        BATCH_SIZE = size_dict[num_digit]

        index = 0
        while index < len(list_data):
            batch = [f"What is {list_data[i]}?" for i in range(index, min(index+BATCH_SIZE, len(list_data)))]
            batch_prompts.append(batch)
            index += BATCH_SIZE
    
print("Number of batches:", len(batch_prompts))
print("Number of prompts in each batch:", [len(batch) for batch in batch_prompts])
            

Number of batches: 380
Number of prompts in each batch: [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 10, 10, 10, 10, 10, 10,

In [6]:
# give a prompt once and then generate the response,
# and then use this prompt+response directly instead of doing everytime

promt_saying_you_are_a_math_assistant = '''You are a math assistant. I will ask you to find number from its word representation. Please answer in the correct format. For example, if I ask 'What is thirty-three?' , you should answer 'thirty-three = 33'. Each question is in a separate line. Please return each answer in a separate line. Do you understand ?'''
Initial_messages = [{'role':'user', 'parts': [promt_saying_you_are_a_math_assistant] }]

# get response from the model and then append it to the prompt
Initial_response = gemini.generate_content(Initial_messages)                                   
Initial_messages.append(Initial_response.candidates[0].content)

In [9]:
print(Initial_messages)

[{'role': 'user', 'parts': ["You are a math assistant. I will ask you to find number from its word representation. Please answer in the correct format. For example, if I ask 'What is thirty-three?' , you should answer 'thirty-three = 33'. Each question is in a separate line. Please return each answer in a separate line. Do you understand ?"]}, parts {
  text: "Yes, I understand. I will find the number from its word representation and answer in the correct format. For example, if you ask \'What is thirty-three?\', I will answer \'thirty-three = 33\'."
}
role: "model"
]


In [15]:
import pickle

# save the initial messages to a file, in case we need to restart the conversation, initial message should be consistent
# across all batches

with open('initial_messages_numeration_word2num.pkl', 'wb') as file:
    pickle.dump(Initial_messages, file)

### conduct experiment 

In [10]:
# path to save responses
response_file = 'gemini_response/numeration_word2num_response.txt'

# request per minute limit for free api
rpm = 60

In [12]:
# to start from a specific batch number, give printed_batch_num -1 here
batch_no_to_start_from = 51 

for i, batch_prompt in enumerate(batch_prompts):

    if i < batch_no_to_start_from:
        continue

    # print status
    print(f"Sending batch {i+1} of {len(batch_prompts)}")

    num_prompts = len(batch_prompt)

    # create prompt for the batch
    curr_prompt = "".join([f"{prompt} \n " for prompt in batch_prompt])

    
    # create a complete prompt to send to the model
    complete_promt = Initial_messages.copy()
    complete_promt.append({'role':'user', 'parts': [curr_prompt]})

    # get response from the model, candidate_count is 1 because we only need one response,
    # temperature is 0 because we need deterministic responses
    response = gemini.generate_content( complete_promt,
                                        generation_config= genai.GenerationConfig(
                                        candidate_count = 1,
                                        temperature = 0.0,
                                        )
                                )
    
    try:
        # extract text from the response
        model_responses = response.candidates[0].content.parts.__getitem__(0).text.split('\n')
    except IndexError:
        time.sleep(5)
        # get response from the model, candidate_count is 1 because we only need one response,
        # temperature is 0 because we need deterministic responses
        response = gemini.generate_content( complete_promt,
                                            generation_config= genai.GenerationConfig(
                                            candidate_count = 1,
                                            temperature = 0.0,
                                            )
                                    )
        
        try:
            # extract text from the response
            model_responses = response.candidates[0].content.parts.__getitem__(0).text.split('\n')
        except IndexError:
            # if model gives no response, fill with 0
            print("Error in response")
            model_responses = ["0"]*num_prompts

    # save responses to file
    with open(response_file, 'a') as file:
        for model_response in model_responses:
            file.write(model_response + '\n')

    
    #print the prompt and response separated by space
    print('# of prompts: ',len(batch_prompt),'# of responses: ', len(model_responses))


    for prompt, response in zip(batch_prompt, model_responses):
        print(prompt, response, sep=" ->")
    
    
    print("Received responses for batch", i+1, "of", len(batch_prompts))
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

    time.sleep(60//rpm) #to avoid rate limit errors

print("Responses saved to", response_file)

Sending batch 52 of 380
# of prompts:  10 # of responses:  10
What is four hundred and thirteen thousand, eight hundred and five? ->four hundred and thirteen thousand, eight hundred and five = 413,805
What is three hundred and twenty-five thousand, eight hundred and fifty-six? ->three hundred and twenty-five thousand, eight hundred and fifty-six = 325,856
What is one hundred and fifty thousand, four hundred and sixty-five? ->one hundred and fifty thousand, four hundred and sixty-five = 150,465
What is five hundred and seventy-eight thousand, nine hundred and sixty-seven? ->five hundred and seventy-eight thousand, nine hundred and sixty-seven = 578,967
What is two hundred and ninety-five thousand, seven hundred and thirty-four? ->two hundred and ninety-five thousand, seven hundred and thirty-four = 295,734
What is three hundred and eleven thousand, six hundred and sixty-nine? ->three hundred and eleven thousand, six hundred and sixty-nine = 311,669
What is two hundred and fifty-nine tho